## LIBRERIAS y CONSTANTES

In [ ]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat
from functions.RL_functions import GetTriplesSubtree, \
                                        GetTreeSize, \
                                        GetAllJoins, \
                                        GetIter, \
                                        GetTotalBgp, \
                                        GetDataframe

from functions.aux import MetricTotalAccuraccy


class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


### Basic parameters

In [ ]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.1_subqueries'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"

#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
min_time_or = 15
max_time_or = 80
#percent_of_data = 1
min_time = 5
max_time = 80
optimizer = "Adam"

In [ ]:
def split_ds(all_data, val_rate, seed):
    """
    Used  to keep a balance of sets with respect to runtime of queries. 
    test_rate is a rate of the total,
    val_rate is a rate of the (total - test_rate)
    :param all_data: Pandas dataframe with data
    :param val_rate: Rate of the (total - test_rate)
    :param seed: For replication of results, this fixes the seed of split method. 
    :return: 
    """
    ranges = {}
    ranges['1_2'] = all_data[(all_data["time"] >= min_time)    & (all_data["time"] <= 2)]
    ranges['2_3'] = all_data[(all_data["time"] > 2)    & (all_data["time"] <= 3)]
    ranges['3_4'] = all_data[(all_data["time"] > 3)    & (all_data["time"] <= 4)]
    ranges['4_5'] = all_data[(all_data["time"] > 4)    & (all_data["time"] <= 5)]
    ranges['5_8'] = all_data[(all_data["time"] > 5)    & (all_data["time"] <= 8)]
    ranges['8_10'] = all_data[(all_data["time"] > 8)   & (all_data["time"] <= 10)]
    ranges['10_20'] =   all_data[(all_data["time"] > 10) & (all_data["time"] <= 20)]
    ranges['20_30'] =   all_data[(all_data["time"] > 20) & (all_data["time"] <= 30)]
    ranges['30_40'] =   all_data[(all_data["time"] > 30) & (all_data["time"] <= 40)]
    ranges['40_50'] =   all_data[(all_data["time"] > 40) & (all_data["time"] <= 50)]
    ranges['50_60'] =   all_data[(all_data["time"] > 50) & (all_data["time"] <= 60)]
    ranges['60_80'] =   all_data[(all_data["time"] > 60) & (all_data["time"] <= 80)]
    ranges['80_100'] =  all_data[(all_data["time"] > 80) & (all_data["time"] <= 100)]
    ranges['100_150'] = all_data[(all_data["time"] > 100) & (all_data["time"] <= 150)]
    ranges['150_200'] = all_data[(all_data["time"] > 150) & (all_data["time"] <= 200)]
    ranges['200_250'] = all_data[(all_data["time"] > 200) & (all_data["time"] <= 250)]
    ranges['250_450'] = all_data[(all_data["time"] > 250) & (all_data["time"] <= 450)]
    ranges['450_last'] = all_data[(all_data["time"] > 450)]
    train_data = []
    val_data = []
    for rang in ranges.values():
        if rang.shape[0] >= 3:
            X_train, X_val = train_test_split(
                rang, test_size=val_rate, shuffle=True,random_state=seed)

            train_data.append(X_train)
            val_data.append(X_val)
    train_data_list = pd.concat(train_data)
    val_data_list = pd.concat(val_data)
    #print("Shapes : Train: {} Val: {}".format(train_data_list.shape, val_data_list.shape))
    return train_data_list, val_data_list
def clear_error_tuples(x):
    try:
        json.loads(x)
        return True
    except:
        print("Error in data ignored!", x)
        return False

### Features

In [ ]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

## CLEAN DATA AND CREATE NEW DATASET TRAIN-TEST

In [ ]:
df_raw = pd.read_csv(URL + csv_name + ".csv", engine='python', encoding='utf-8')

In [ ]:
print("df_raw.shape", df_raw.shape)
df_raw['time'].value_counts(bins=100, sort=True, normalize=True)

In [ ]:
df_raw['time'].describe()

In [ ]:
if 'sparql_file' in df_raw.columns:
    df_raw = df_raw.rename(columns = {'sparql_file': 'query'}, inplace = False)
df_raw['cpu_p'] = df_raw['cpu_p'].apply(lambda x: float(x.strip('%')))
df_raw['same_seg_p'] = df_raw['same_seg_p'].apply(lambda x: float(x.strip('%')))
df_raw['same_page_p'] = df_raw['same_page_p'].apply(lambda x: float(x.strip('%')))
df_raw['wait'] = df_raw['wait'].apply(lambda x: float(x.strip('%')))
df_raw['comp_read_p'] = df_raw['comp_read_p'].apply(lambda x: float(x.strip('%')))
df_raw['comp_clw'] = df_raw['comp_clw'].apply(lambda x: float(x.strip('%')))

In [ ]:
print("mean", df_raw['time'].mean())
print("std", df_raw['time'].std())
print('df_raw.shape',df_raw.shape)
print("max", df_raw['time'].max())
print("df_raw.shape", df_raw.shape)
df_raw = df_raw[df_raw['time'] <= max_time_or]
df_raw = df_raw[df_raw['time'] >= min_time_or]
df_raw = df_raw.reset_index(drop=True)
print("CLEAN by TIME")
print("mean", df_raw['time'].mean())
print("std", df_raw['time'].std())
print("max", df_raw['time'].max())
print("df_raw.shape", df_raw.shape)

In [ ]:
df_raw['time'].describe()

In [ ]:
ds_model, ds_rl_prev = split_ds(df_raw, 0.15,seed=None)

In [ ]:
print("ds_model.shape",ds_model.shape)
print("ds_rl_prev.shape",ds_rl_prev.shape)

In [ ]:
ds_train_val_prev, ds_test_prev = split_ds(ds_model, 0.2,seed=None)
ds_train_prev, ds_val_prev = split_ds(ds_train_val_prev, 0.25,seed=None)
print("ds_train_val_prev.shape",ds_train_val_prev.shape)
print("ds_test_prev.shape",ds_test_prev.shape)
print("ds_train_prev.shape",ds_train_prev.shape)
print("ds_val_prev.shape",ds_val_prev.shape)

In [ ]:
def subtree_format(df_raw):
    df_raw_unique_id = df_raw['unique_id']
    df_raw_filename = df_raw['filename']
    df_raw_query = df_raw['query']
    df_raw_bgp = df_raw['bgps']
    df_raw_json_cardinality = df_raw['json_cardinality']
    df_raw_subtrees = df_raw['matrix_subtrees']
    columns = ['unique_id', 'filename', 'query', 'trees',  'bgps' ,'time', 'total_bgps', 'triples', 'treesize', 'join', 'left_join', 'iter', 'json_cardinality_original_query']
    values = []
    for dfrs in range(0,len(df_raw_subtrees)):
        unique_id = df_raw_unique_id[dfrs]
        filename = df_raw_filename[dfrs]
        query = df_raw_query[dfrs]
        bgp = df_raw_bgp[dfrs]
        json_cardinality = df_raw_json_cardinality[dfrs]
        lists_type = ast.literal_eval(df_raw_subtrees[dfrs])
        for ls in lists_type:
            str_subtree = str(ls[0]).replace('"', ';').replace("'", '"')
            row = [unique_id, filename, query, str_subtree, bgp] + ls[1:] + [json_cardinality]
            values.append(row)
            
    df_subtrees = pd.DataFrame(values, columns=columns)
    return df_subtrees
    

In [ ]:
def json_cardinality_subtree(df_subtrees):
    dfsq_unique_id = df_subtrees['unique_id']
    dfsq_filename = df_subtrees['filename']
    dfsq_query = df_subtrees['query']
    dfsq_trees = df_subtrees['trees']
    dfsq_time = df_subtrees['time']
    dfsq_total_bgps = df_subtrees['total_bgps']
    dfsq_triples = df_subtrees['triples']
    dfsq_treesize = df_subtrees['treesize']
    dfsq_join = df_subtrees['join']
    dfsq_left_join = df_subtrees['left_join']
    dfsq_iter = df_subtrees['iter']
    dfsq_json_cardinality_original = df_subtrees['json_cardinality_original_query']
    columns = ['unique_id', 'filename', 'query', 'trees', 'time', 'total_bgps', 'triples', 'treesize', 'join', 'left_join', 'iter', 'json_cardinality_original_query', 'json_cardinality']
    values = []
    for df in range(len(dfsq_trees)):
        json_cardinality = {}
        tree_as_str = str(dfsq_trees[df])
        json_cardinality_original = ast.literal_eval(dfsq_json_cardinality_original[df])
        for k,v in json_cardinality_original.items():
            #fix = k.replace(';','"')
            if k in tree_as_str:
                json_cardinality[str(k)] = str(v)
        values.append(str(json_cardinality).replace('"', ';').replace("'", '"'))  
    df_subtrees['json_cardinality'] = values
    return df_subtrees

In [ ]:
ds_train_prev = ds_train_prev.reset_index()
ds_val_prev = ds_val_prev.reset_index()
ds_test_prev = ds_test_prev.reset_index()
ds_rl_prev = ds_rl_prev.reset_index()

ds_train_prev_subtree = subtree_format(ds_train_prev)
ds_val_prev_subtree = subtree_format(ds_val_prev)
ds_test_prev_subtree = subtree_format(ds_test_prev)
ds_rl_prev_subtree = subtree_format(ds_rl_prev)

ds_train_prev_subtree['time'] = ds_train_prev_subtree.time.astype(float)
ds_val_prev_subtree['time'] = ds_val_prev_subtree.time.astype(float)
ds_test_prev_subtree['time'] = ds_test_prev_subtree.time.astype(float)
ds_rl_prev_subtree['time'] = ds_rl_prev_subtree.time.astype(float)

#ds_train_prev_subtree = json_cardinality_subtree(ds_train_prev_subtree)
#ds_val_prev_subtree = json_cardinality_subtree(ds_val_prev_subtree)
#ds_test_prev_subtree = json_cardinality_subtree(ds_test_prev_subtree)
#ds_rl_prev_subtree = json_cardinality_subtree(ds_rl_prev_subtree)

In [ ]:
def define_time_limit(df_st, max_time, min_time, printing=True):
    if printing:
        print("+++++++++++++++++++++++++++++++")
        print("+++++++++++++++++++++++++++++++")
        print("+++++++++++++++++++++++++++++++")
        print("---------NO CLEAN------------")
        print(df_st['time'].describe())
        print("-----------------------------")
        print("mean", df_st['time'].mean())
        print("std", df_st['time'].std())
        print('shape',df_st.shape)
        print("max", df_st['time'].max())
    df_st = df_st[df_st['time'] <= max_time]
    df_st = df_st[df_st['time'] >= min_time]
    df_st = df_st.reset_index(drop=True)
    if printing:
        print("--------FIRST CLEAN-------------")
        print(df_st['time'].describe())
        print("--------------------------------")
        print("mean", df_st['time'].mean())
        print("std", df_st['time'].std())
        print('shape',df_st.shape)
        print("max", df_st['time'].max())
    return df_st

In [ ]:
printing = True
print("++++++++++++++++TRAIN++++++++++++++++++")
ds_train_prev_subtree = define_time_limit(ds_train_prev_subtree, max_time, min_time, printing)
print("++++++++++++++++VALIDATION++++++++++++++++++")
ds_val_prev_subtree = define_time_limit(ds_val_prev_subtree, max_time, min_time, printing)
print("++++++++++++++++TEST++++++++++++++++++")
ds_test_prev_subtree = define_time_limit(ds_test_prev_subtree, max_time, min_time, printing)
print("++++++++++++++++RL++++++++++++++++++")
ds_rl_prev_subtree = define_time_limit(ds_rl_prev_subtree, max_time, min_time, printing)

In [ ]:
#Remove bad rows
ds_train  = ds_train_prev_subtree[ds_train_prev_subtree['trees'].apply(lambda x: clear_error_tuples(x))]
ds_val  = ds_val_prev_subtree[ds_val_prev_subtree['trees'].apply(lambda x: clear_error_tuples(x))]
ds_test  = ds_test_prev_subtree[ds_test_prev_subtree['trees'].apply(lambda x: clear_error_tuples(x))]
ds_rl = ds_rl_prev_subtree[ds_rl_prev_subtree['trees'].apply(lambda x: clear_error_tuples(x))]

print("---------SHAPES-----------")
print("----------RAW-----------")
print(f'shape df_raw: {df_raw.shape}')
print("----------PREV----------")
print(f'shape ds_train_val_prev: {ds_train_val_prev.shape}')
print(f'shape ds_train_prev: {ds_train_prev.shape}')
print(f'shape ds_val_prev: {ds_val_prev.shape}')
print(f'shape ds_test_prev: {ds_test_prev.shape}')
print(f'shape ds_rl_prev: {ds_rl_prev.shape}')
print("----------CLEAN----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val: {ds_val.shape}')
print(f'shape ds_test: {ds_test.shape}')
print(f'shape ds_rl: {ds_rl.shape}')

In [ ]:
ds_rl.columns

In [ ]:
#if active_new_data:
#    ds_train.to_csv(URL + csv_name + '_ds_train.csv')
#    ds_val.to_csv(URL + csv_name + '_ds_val.csv')
#    ds_test.to_csv(URL + csv_name + '_ds_test.csv')
#    ds_rl.to_csv(URL + csv_name + '_ds_rl.csv')
#    print("New csv generates")
#else:
#    print("Not csv generates")

# FUNCTIONS

In [ ]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [ ]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

In [ ]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [ ]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


In [ ]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

## NeoRegression
Esta en model_trees_algebra.py

In [ ]:
aec_dir = ''
learning_rate = 0.00010
batch_size = 64
verbose=True
reg = NeoRegression(
     aec={'train_aec': False, 'use_aec': True,'aec_file': '', 'aec_epochs': 200},
     epochs=400,
     maxcardinality=maxcardinality,
     in_channels_neo_net=64,
     tree_units=[64,32],
     tree_units_dense=[16],
     early_stop_patience=10,
     early_stop_initial_patience=180,
     tree_activation_tree=nn.LeakyReLU,
     tree_activation_dense=nn.ReLU,
     optimizer={'optimizer': optimizer,'args':{"lr":learning_rate}},
     figimage_size=(18,18),
     start_history_from_epoch=3,
     batch_size=batch_size
)


In [ ]:
from IPython.utils import io

In [ ]:
#Fit the transformer tree data
reg.fit_transform_tree_data(ds_train, ds_val, ds_test, ds_rl)
print("Trees tranformed!!!")


In [ ]:
#with io.capture_output() as captured:
#x_train_query, x_val_query, x_test_query, x_rl_query =  prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query)
x_train_query, x_val_query, x_test_query, x_rl_query =  prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query)
print("END PREPARE QUERY LEVEL DATA")

In [ ]:
y_train = y_train.astype(float)
y_val = y_val.astype(float)

In [ ]:
#Fit model
reg.fit(x_train_tree, x_train_query.values, y_train, x_val_tree, x_val_query.values, y_val)

#### SAVE MODEL

In [ ]:
#Save best model
import torch
torch.save(reg.best_model.state_dict(), "./best_model_fullADAM-7.0_V3_el_mejor_hasta_ahora.pt")
torch.save(reg.best_model.state_dict(), "./best_model_brigido.pt")
#Save stats in val set
file_to_store = open("./execution_model_stats_fullADAM-7.0_V3_el_mejor_hasta_ahora.pickle", "wb")
pickle.dump(getpredictions_info_nojc(x_val_tree, x_val_query, y_val), file_to_store)
file_to_store.close()


#### TESTING

In [ ]:
#val_stats = getpredictions_info(x_val_tree, x_val_query, y_val)
val_stats = getpredictions_info_nojc(x_val_tree, x_val_query, y_val)

In [ ]:
ds_val['y_pred'] = val_stats['pred']

In [ ]:
test_stats = getpredictions_info_nojc(x_test_tree, x_test_query, y_test)

In [ ]:
test_stats.keys()

In [ ]:
ds_test['y_realcheck'] = test_stats['real']
ds_test['y_pred'] = test_stats['pred']

In [ ]:
def tag_points(x):
    """Add quality tags of predictions. Used to plot with plotly"""
    difference = x['time'] - x['y_pred'][0]
    abs_diff = np.abs(difference)
    x['y_pred'] = x['y_pred'][0]
    x['query2'] = x['query'].replace(" . ", ' . <br>').replace(" FILTER", '<br> FILTER').replace(" { ", ' { <br>').replace(" } ", ' <br> }').replace(" ; ", ' ; <br>') 
    p20 = x['time'] * 0.2
    p40 = x['time'] * 0.4
    if abs_diff < p20:
        x['color'] = "good prediction"
    elif abs_diff < p40:
        x['color'] = "aceptable prediction"
    else:
        x['color'] = "bad prediction"
    return x

In [ ]:
def tag_points2(x):
    """Add quality tags of predictions. Used to plot with plotly"""
    difference = x['time'] - x['y_realcheck'][0]
    abs_diff = np.abs(difference)
    x['y_realcheck'] = x['y_realcheck'][0]
    x['query2'] = x['query'].replace(" . ", ' . <br>').replace(" FILTER", '<br> FILTER').replace(" { ", ' { <br>').replace(" } ", ' <br> }').replace(" ; ", ' ; <br>') 
    p20 = x['time'] * 0.2
    p40 = x['time'] * 0.4
    if abs_diff < p20:
        x['color'] = "good prediction"
    elif abs_diff < p40:
        x['color'] = "aceptable prediction"
    else:
        x['color'] = "bad prediction"
    return x

In [ ]:
#tag_points(ds_test.values[0])
other = ds_test.apply(lambda x: tag_points(x), axis=1)

In [ ]:
other.to_pickle("./predictions_test.pickle")

In [ ]:
otherval = ds_val.apply(lambda x: tag_points(x), axis=1)

In [ ]:
otherval.to_pickle("./predictions_val.pickle")

In [ ]:

fig = px.scatter(otherval[['query','query2','time','y_pred','color']], x="y_pred", y="time", color="color", hover_data=['query2'])
fig.update_layout(height=800, width=1000, title_text="Predictions on Val Set")
fig.show()


In [ ]:

fig = px.scatter(other[['query','query2','time','y_pred','y_realcheck','color']], x="y_pred", y="time", color="color", hover_data=['query2'])
fig.update_layout(height=800, width=1000, title_text="Predictions on Test Set")
fig.show()


In [ ]:
tot_val,b_val,a_val,g_val,bp_val,ap_val,gp_val = MetricTotalAccuraccy(otherval)
print(f"Total predictions: {tot_val}")
print(f"Bad predictions: {b_val}, percentage {bp_val}%")
print(f"Acceptable predictions: {a_val}, percentage {ap_val}%")
print(f"Good predictions: {g_val}, percentage {gp_val}%")

print(f"Accuraccy: {100*(a_val+g_val)/tot_val}%")

In [ ]:
tot_val,b_val,a_val,g_val,bp_val,ap_val,gp_val = MetricTotalAccuraccy(other)
print(f"Total predictions: {tot_val}")
print(f"Bad predictions: {b_val}, percentage {bp_val}%")
print(f"Acceptable predictions: {a_val}, percentage {ap_val}%")
print(f"Good predictions: {g_val}, percentage {gp_val}%")

print(f"Accuraccy: {100*(a_val+g_val)/tot_val}%")

## Reinforcement Learning

In [ ]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num > epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = getpredictions_info_nojc(new_tree, x_rl_query, y_rl_ind)['pred']
    reward -= pred[0][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num > epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = getpredictions_info_nojc(new_tree, x_rl_query, y_rl_ind)['pred']
        reward -= pred[0][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        q_val[act[0]] = max(q_val[act[0]],reward)
    return q_value_neo

In [ ]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

### RL Parameters

In [ ]:
buenos_idx = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= 5:
        buenos_idx.append(i)
random_value = random.choice(buenos_idx)
random_value = 1309
print(buenos_idx)
print(len(buenos_idx))

## RL Loop

### RL Loop - 1 Val - General Features

In [ ]:
x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,random_value)
actions = RL_Actions(bgp_ind)
num_actions = len(actions)
print("actions")
print(actions)
print("num_actions", num_actions)
print("-------")

### Q-Learning - 1 Loop

##### Parameters

In [ ]:
q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
epsilon_ql = 0.55 ## EPSILON
lr_ql = 0.9 ## LEARNING RATE
disc_fac_ql = 0.9 ## DISCOUNT FACTOR
epoch_ql = 100 ### TOTAL EPOCH

##### Loop

In [ ]:
values_ql = []
for i in range(0,epoch_ql):
    current_state = []
    reward = 0
    terminal = False
    current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                    current_state,
                                                                                    reward,
                                                                                    q_value_ql,
                                                                                    x_rl_tree_ind,
                                                                                    x_rl_query_ind,
                                                                                    y_rl_ind,
                                                                                    bgp_ind,
                                                                                    epsilon_ql)
    old_state = len(current_state)-1
    prev_action = chosen_action_idx
    while not terminal:
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        
        new_state = len(current_state)-1
        if not terminal:
            old_q_value = q_value_ql[old_state,prev_action]
            temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
        old_state = new_state
        prev_action = chosen_action_idx
    old_q_value = q_value_ql[old_state,prev_action]
    new_q_value = old_q_value + lr_ql * (reward - old_q_value)
    q_value_ql[old_state,prev_action] = new_q_value
    best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
    rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
    rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
    row = [
                    random_value,
                    i,
                    rl_stats_old['pred'][0][0],
                    rl_stats_new['pred'][0][0],
                    float(rl_stats_old['real'][0][0]),
                    float(rl_stats_new['real'][0][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    rl_stats_old['history'],
                    rl_stats_new['history'],
           ]

    values_ql.append(row)

In [ ]:
best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
print("best_state \n",best_state)
print("new_dicto \n",new_dicto)
print("new_tree \n",new_tree)
print("old_tree \n", x_rl_tree_ind)
print("\n")
print("VALORES CON VIEJO ARBOL")
rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_old['pred'][0][0])
print("valor real: ",float(rl_stats_old['real'][0][0]))

print("VALORES CON NUEVO ARBOL")
rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_new['pred'][0][0])
print("valor real: ",float(rl_stats_new['real'][0][0]))

In [ ]:
values_ql[0]

### Sarsa

##### Parameters

In [ ]:
q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
epsilon_sar = 0.75 ## EPSILON
lr_sar = 0.9 ## LEARNING RATE
disc_fac_sar = 0.9 ## DISCOUNT FACTOR
epoch_sar = 1000 ### TOTAL EPOCH

##### Loop 1 Val

In [ ]:
for i in range(0,epoch_sar):
    current_state = []
    reward = 0
    terminal = False
    current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                    current_state,
                                                                                    reward,
                                                                                    q_value_sar,
                                                                                    x_rl_tree_ind,
                                                                                    x_rl_query_ind,
                                                                                    y_rl_ind,
                                                                                    bgp_ind,
                                                                                    epsilon_sar)
    old_state = len(current_state)-1
    prev_action = chosen_action_idx
    while not terminal:
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        
        new_state = len(current_state)-1
        if not terminal:
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
        old_state = new_state
        prev_action = chosen_action_idx
    old_q_value = q_value_sar[old_state,prev_action]
    new_q_value = old_q_value + lr_sar * (reward - old_q_value)
    q_value_sar[old_state,prev_action] = new_q_value

In [ ]:
best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
print("best_state \n",best_state)
print("new_dicto \n",new_dicto)
print("new_tree \n",new_tree)
print("old_tree \n", x_rl_tree_ind)
print("\n")
print("VALORES CON VIEJO ARBOL")
rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_old['pred'][0][0])
print("valor real: ",int(rl_stats_old['real'][0][0]))

print("VALORES CON NUEVO ARBOL")
rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_new['pred'][0][0])
print("valor real: ",int(rl_stats_new['real'][0][0]))

## Expected Sarsa

##### Parameters

In [ ]:
q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
epsilon_exsar = 0.75 ## EPSILON
lr_exsar = 0.9 ## LEARNING RATE
disc_fac_exsar = 0.9 ## DISCOUNT FACTOR
epoch_exsar = 1000 ### TOTAL EPOCH

##### Loop 1 Val

In [ ]:
for i in range(0,epoch_exsar):
    current_state = []
    reward = 0
    terminal = False
    current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                    current_state,
                                                                                    reward,
                                                                                    q_value_exsar,
                                                                                    x_rl_tree_ind,
                                                                                    x_rl_query_ind,
                                                                                    y_rl_ind,
                                                                                    bgp_ind,
                                                                                    epsilon_exsar)    
    old_state = len(current_state)-1
    prev_action = chosen_action_idx
    while not terminal:
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)
        new_state = len(current_state)-1
        array_state = np.array(current_state)
        
        
        
        
        expected_q = 0
        old_q_value = q_value_exsar[old_state,prev_action]
        
        q_max = np.max(q_value_exsar[new_state,:])
        pi = np.ones(num_actions)*epsilon_exsar/num_actions \
            + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar)/np.sum(q_value_exsar[new_state,:] == q_max)
        
        expected_q = np.sum(q_value_exsar[new_state,:]*pi)
        temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
        new_q_value = old_q_value +  lr_exsar*temporal_difference
        q_value_exsar[old_state,prev_action] = new_q_value

        old_state = new_state
        prev_action = chosen_action_idx
    old_q_value = q_value_exsar[old_state,prev_action]
    new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
    q_value_exsar[old_state,prev_action] = new_q_value
print(q_value_exsar)

In [ ]:
best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
print("best_state \n",best_state)
print("new_dicto \n",new_dicto)
print("new_tree \n",new_tree)
print("old_tree \n", x_rl_tree_ind)
print("\n")
print("VALORES CON VIEJO ARBOL")
rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_old['pred'][0][0])
print("valor real: ",int(rl_stats_old['real'][0][0]))

print("VALORES CON NUEVO ARBOL")
rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_new['pred'][0][0])
print("valor real: ",int(rl_stats_new['real'][0][0]))

## RL-Neo

##### Parameters

In [ ]:
q_value_neo = RL_First_Policy(actions)-99999 ##INITIAL POLICITY
epsilon_neo = 0.55 ## EPSILON
epoch_neo = 1000 ### TOTAL EPOCH

##### Loop 1 Val

In [ ]:
for i in range(0,epoch_neo):
    current_state = []
    reward = 0
    terminal = False
    current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                    current_state,
                                                                                    reward,
                                                                                    q_value_neo,
                                                                                    x_rl_tree_ind,
                                                                                    x_rl_query_ind,
                                                                                    y_rl_ind,
                                                                                    bgp_ind,
                                                                                    epsilon_neo)    
    old_state = len(current_state)-1
    while not terminal:
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)
    q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)


In [ ]:
best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
print("best_state \n",best_state)
print("new_dicto \n",new_dicto)
print("new_tree \n",new_tree)
print("old_tree \n", x_rl_tree_ind)
print("\n")
print("VALORES CON VIEJO ARBOL")
rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_old['pred'][0][0])
print("valor real: ",int(rl_stats_old['real'][0][0]))

print("VALORES CON NUEVO ARBOL")
rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
print("valor predict: ",rl_stats_new['pred'][0][0])
print("valor real: ",int(rl_stats_new['real'][0][0]))

## MULTI-VAL

##### Seleccionar data con mas de 1 o más acciones y parámetros generales

In [ ]:
min_num_actions = -1
buenos_idx = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_buenos_idx",total_buenos_idx)
print("buenos_idx")
print(buenos_idx)

### Q - Learning - multival

##### Parameters

In [ ]:
epsilon_ql = 0.75 ## EPSILON
lr_ql = 0.9 ## LEARNING RATE
disc_fac_ql = 0.9 ## DISCOUNT FACTOR
epoch_ql = 100 ### TOTAL EPOCH POR DATA
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'history_old',
            'history_new'
             ]

##### Loop

In [ ]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)

            new_state = len(current_state)-1
            if not terminal:
                old_q_value = q_value_ql[old_state,prev_action]
                temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
                new_q_value = old_q_value + lr_ql * temporal_difference
                q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0][0],
                    rl_stats_new['pred'][0][0],
                    float(rl_stats_old['real'][0][0]),
                    float(rl_stats_new['real'][0][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    rl_stats_old['history'],
                    rl_stats_new['history'],
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

In [ ]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_no_history = ql_df[['index','iteration','pred_old','pred_new','real_old','real_new','mse_old','mse_new','mae_old','mae_new','rmse_old','rmse_new']]
ql_df_no_history.to_csv(URL + 'rl_csvs/ql_df_no_history.csv',index=False)

In [ ]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final.csv',index=False)

## RL-Neo Multival

###### Parameters

In [ ]:
epsilon_neo = 0.75 ## EPSILON
epoch_neo = 100 ### TOTAL EPOCH POR DATA

###### Loop

In [ ]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        old_state = len(current_state)-1
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0][0],
                    rl_stats_new['pred'][0][0],
                    float(rl_stats_old['real'][0][0]),
                    float(rl_stats_new['real'][0][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    rl_stats_old['history'],
                    rl_stats_new['history'],
           ]

        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

In [ ]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_no_history = neo_df[['index','iteration','pred_old','pred_new','real_old','real_new','mse_old','mse_new','mae_old','mae_new','rmse_old','rmse_new']]
neo_df_no_history.to_csv(URL + 'rl_csvs/neo_df_no_history.csv',index=False)

In [ ]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final.csv',index=False)

## Sarsa Multival

###### Parameters

In [ ]:
q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
epsilon_sar = 0.75 ## EPSILON
lr_sar = 0.9 ## LEARNING RATE
disc_fac_sar = 0.9 ## DISCOUNT FACTOR
epoch_sar = 100 ### TOTAL EPOCH

###### Loop

In [ ]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            if not terminal:
                old_q_value = q_value_sar[old_state,prev_action]
                temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
                new_q_value = old_q_value + lr_sar * temporal_difference
                q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0][0],
                    rl_stats_new['pred'][0][0],
                    float(rl_stats_old['real'][0][0]),
                    float(rl_stats_new['real'][0][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    rl_stats_old['history'],
                    rl_stats_new['history'],
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

In [ ]:
sarsa_df = pd.DataFrame(values_sarsa,columns=rl_columns)
sarsa_df_no_history = sarsa_df[['index','iteration','pred_old','pred_new','real_old','real_new','mse_old','mse_new','mae_old','mae_new','rmse_old','rmse_new']]
sarsa_df_no_history.to_csv(URL + 'rl_csvs/sarsa_df_no_history.csv',index=False)

In [ ]:
sarsa_df_final = pd.DataFrame(values_final_sarsa,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final.csv',index=False)

## Expected-Sarsa Multival

###### Parameters

In [ ]:
#q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
epsilon_exsar = 0.75 ## EPSILON
lr_exsar = 0.9 ## LEARNING RATE
disc_fac_exsar = 0.9 ## DISCOUNT FACTOR
epoch_exsar = 100 ### TOTAL EPOCH

###### Loop

In [ ]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]

            q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = getpredictions_info_nojc(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = getpredictions_info_nojc(new_tree, x_rl_query_ind, y_rl_ind)
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0][0],
                    rl_stats_new['pred'][0][0],
                    float(rl_stats_old['real'][0][0]),
                    float(rl_stats_new['real'][0][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    rl_stats_old['history'],
                    rl_stats_new['history'],
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

In [ ]:
exsarsa_df = pd.DataFrame(values_exsarsa,columns=rl_columns)
exsarsa_df_no_history = exsarsa_df[['index','iteration','pred_old','pred_new','real_old','real_new','mse_old','mse_new','mae_old','mae_new','rmse_old','rmse_new']]
exsarsa_df_no_history.to_csv(URL + 'rl_csvs/exsarsa_df_no_history.csv',index=False)

In [ ]:
exsarsa_df_final = pd.DataFrame(values_final_exsarsa,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final.csv',index=False)

### Archivos enteros

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)